In [1]:
import numpy as np
import os 
import h5py
import scipy.io

In [2]:
data_dir = '/data/cees/gegewen/'

In [3]:
# load data S
S_data = scipy.io.loadmat('/data/cees/gegewen/cs231n_project_data/sat_map_v2.mat')['sat_map']
print('S_data shape', S_data.shape)


S_data shape (16384, 4, 50, 50)


In [4]:
S = np.concatenate([S_data[:,:,:24,:], S_data[:,:,32:,:]], axis = 2)
print('S shape', S.shape)
S = S.reshape(128, 128, 4, 42, 50)
print('S shape', S.shape)

S shape (16384, 4, 42, 50)
S shape (128, 128, 4, 42, 50)


In [5]:
# load data inj
inj_data = scipy.io.loadmat('/data/cees/gegewen/cs231n_project_data/inj_map_v2.mat')['inj_map']
print('inj_data shape', inj_data.shape)

inj_data shape (128, 128, 50, 50)


In [6]:
inj = np.concatenate([inj_data[:,:,:,:24], inj_data[:,:,:,32:]], axis = 3)
print('inj shape', inj.shape)
inj = inj.swapaxes(2,3)
print('inj shape', inj.shape)

inj shape (128, 128, 50, 42)
inj shape (128, 128, 42, 50)


In [7]:
#load data k
k_data = scipy.io.loadmat('/data/cees/gegewen/cs231n_project_data/kmap.mat')['kmap']
print('k_data shape', k_data.shape)
k = np.concatenate([k_data[:,:24], k_data[:,32:50]], axis = 1)
print('k shape', k.shape)

KeyboardInterrupt: 

In [ ]:
k = k.reshape(128, 128, 42)
print('k shape', k.shape)

In [8]:
# load data P
P_data = scipy.io.loadmat('/data/cees/gegewen/cs231n_project_data/pressure_map_v2.mat')['pressure_map']
print('Pressure_data shape', P_data.shape)

Pressure_data shape (16384, 4, 50, 50)


In [9]:
P = np.concatenate([P_data[:,:,:24,:], P_data[:,:,32:,:]], axis = 2)
print('P shape', P.shape)
P = P.reshape(128, 128, 4, 42, 50)
print('P shape', P.shape)

P shape (16384, 4, 42, 50)
P shape (128, 128, 4, 42, 50)


In [10]:
# reshape data S
S = S.reshape(128, 128, 4, 42, 50, 1)
print('S shape', S.shape)

S shape (128, 128, 4, 42, 50, 1)


In [11]:
# reshape data inj
print('inj shape', inj.shape)
inj = np.repeat(inj[:, :, np.newaxis, :, :], 4, axis = 2)
print('inj shape', inj.shape)
inj = inj.reshape(128, 128, 4, 42, 50, 1)
print('inj shape', inj.shape)
inj = inj.swapaxes(0,1)

inj shape (128, 128, 42, 50)
inj shape (128, 128, 4, 42, 50)
inj shape (128, 128, 4, 42, 50, 1)


In [12]:
inj_day = np.zeros_like(inj)
inj_day[:, :, 0, :, :, :] = inj[:, :, 0, :, :, :].astype('float64') * 25
inj_day[:, :, 1, :, :, :] = inj[:, :, 1, :, :, :].astype('float64') * 50
inj_day[:, :, 2, :, :, :] = inj[:, :, 2, :, :, :].astype('float64') * 75
inj_day[:, :, 3, :, :, :] = inj[:, :, 3, :, :, :].astype('float64') * 100

In [13]:
# reshape data P
P = P.reshape(128, 128, 4, 42, 50, 1)
print('P shape', P.shape)

P shape (128, 128, 4, 42, 50, 1)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt 

day_idx = 3
k_idx = 4

for well_idx in range (5,10):
    plt.set_cmap('jet')

    plt.figure(figsize=(12, 3))
    plt.subplot(1,3,1)
    plt.imshow(S[:, :, day_idx, k_idx,well_idx,0])
    plt.title('SG')
    plt.plot()
    plt.colorbar(fraction=0.046)
    
    plt.subplot(1,3,2)
    plt.imshow(inj_day[:, :, day_idx, k_idx, well_idx, 0])
    plt.title('inj (day)')
    plt.plot()
    plt.colorbar(fraction=0.046)

    plt.subplot(1,3,3)
    plt.imshow(P[:, :, day_idx, k_idx, well_idx, 0])
    plt.title('P')
    plt.plot()
    plt.colorbar(fraction=0.046)
    
    plt.tight_layout()
    plt.show()


In [15]:
print('P shape', P.shape)
print('S shape', S.shape)
print('inj_day shape', inj_day.shape)

P shape (128, 128, 4, 42, 50, 1)
S shape (128, 128, 4, 42, 50, 1)
inj_day shape (128, 128, 4, 42, 50, 1)


In [16]:
inj_S_P = np.concatenate([inj_day, S, P], axis = 5)
print('package shape is ', inj_S_P.shape)

package shape is  (128, 128, 4, 42, 50, 3)


In [51]:
inj_S_P_1 = inj_S_P[..., :21, :, :].reshape(128, 128, 4, 21, 50 ,3, 1)
inj_S_P_2 = inj_S_P[..., 21:, :,:].reshape(128, 128, 4, 21, 50 ,3, 1)
print('package1 shape is ', inj_S_P_1.shape)
print('package2 shape is ', inj_S_P_2.shape)

package1 shape is  (128, 128, 4, 21, 50, 3, 1)
package2 shape is  (128, 128, 4, 21, 50, 3, 1)


In [53]:
inj_S_P_data = np.concatenate([inj_S_P_1, inj_S_P_2], axis = 6)
print('package shape is ', inj_S_P_data.shape)
inj_S_P_data = inj_S_P_data.swapaxes(3,4)
print('package shape is ', inj_S_P_data.shape)

package shape is  (128, 128, 4, 21, 50, 3, 2)
package shape is  (128, 128, 4, 50, 21, 3, 2)


In [17]:
hw = h5py.File('/data/cees/gegewen/cs231n_project_data/inj_S_P_data_v3.hdf5', 'w')
hw.create_dataset('inj_S_P_data', data = inj_S_P)
hw.close()